In [30]:
import cartopy.crs as ccrs # used for map projection
import matplotlib.pyplot as plt # matplotlib
import cartopy.feature as cfeature # used for map projection
import xarray as xr # x-array
import numpy as np # numpy
import urllib.request # download request
import warnings # to suppress warnings
from numpy import linalg as LA # to plot the moments (by calculating the eigenvalues)
from sklearn.cluster import k_means # to perform k-means
from collections import Counter # set operations
import pandas as pd
warnings.filterwarnings('ignore')
obs = pd.read_csv('data/ObsSST_processed.csv')

In [31]:
def map_background(label=False, extent=[-100, 0, 0, 60]):
  # A helpder function for creating the map background.
  # INPUT:
  # "extent": corresponds to the location information of the showed map.
  # "label": boolean

  # OUTPUT:
  # Matplotlib AXES object

  plt.figure(figsize = (20, 10))
  ax = plt.axes(projection=ccrs.PlateCarree())
  ax.coastlines()
  ax.set_extent(extent)
  ax.gridlines(draw_labels=label) # show labels or not
  LAND = cfeature.NaturalEarthFeature('physical', 'land', '10m',
                                      edgecolor='face',
                                      facecolor=cfeature.COLORS['land'],
                                          linewidth=.1)
  OCEAN = cfeature.NaturalEarthFeature('physical', 'ocean', '10m',
                                       edgecolor='face',
                                       facecolor=cfeature.COLORS['water'], linewidth=.1)
  ax.add_feature(LAND, zorder=0)
  ax.add_feature(OCEAN)
  return ax

In [32]:
tks = xr.open_dataset('data/NA_data.nc', engine="netcdf4", decode_times=False)

## Print the tks to take a peek at what's actually in the dataset.
print(tks)

<xarray.Dataset> Size: 618MB
Dimensions:           (storm: 2344, date_time: 360, quadrant: 4)
Coordinates:
    time              (storm, date_time) float64 7MB ...
    lat               (storm, date_time) float32 3MB ...
    lon               (storm, date_time) float32 3MB ...
Dimensions without coordinates: storm, date_time, quadrant
Data variables: (12/147)
    numobs            (storm) float32 9kB ...
    sid               (storm) |S13 30kB ...
    season            (storm) float32 9kB ...
    number            (storm) int16 5kB ...
    basin             (storm, date_time) |S2 2MB ...
    subbasin          (storm, date_time) |S2 2MB ...
    ...                ...
    reunion_gust      (storm, date_time) float32 3MB ...
    reunion_gust_per  (storm, date_time) float32 3MB ...
    usa_seahgt        (storm, date_time) float32 3MB ...
    usa_searad        (storm, date_time, quadrant) float32 14MB ...
    storm_speed       (storm, date_time) float32 3MB ...
    storm_dir         (storm,

In [33]:
storm_1 = tks.sel(storm=2000) # notice that the first storm starts with 0 not 1
storm_2 = tks.sel(storm=2001)
name_1 = str(storm_1.name.values)[11:-1] # use this trick to obtain the name string
name_2 = str(storm_2.name.values)[11:-1] # use this trick to obtain the name string
sid_1 = str(storm_1.sid.values,'UTF-8')
sid_2 = str(storm_2.sid.values,'UTF-8')
print(f"\nThe 2001st hurricane, named {name_1}, has a record of {int(storm_1.numobs)} \
observations with sid {sid_1} in year {int(storm_1.season)}.")
print(f"\nThe 2002nd hurricane, named {name_2}, has a record of {int(storm_2.numobs)} \
observations with sid {sid_2} in year {int(storm_2.season)}.")


The 2001st hurricane, named 'ARLENE', has a record of 48 observations with sid 2005160N17276 in year 2005.

The 2002nd hurricane, named 'BRET', has a record of 12 observations with sid 2005180N20265 in year 2005.


In [34]:
# Create DataFrames
storm_1_data = {
    "Timestamp": storm_1.iso_time.values.astype(str),
    "Latitude": storm_1.lat.values,
    "Longitude": storm_1.lon.values,
    "Wind Speed (wmo)": storm_1.wmo_wind.values,
    "USA Status (wmo)": storm_1.usa_status.values
}

storm_2_data = {
    "Timestamp": storm_2.iso_time.values.astype(str),
    "Latitude": storm_2.lat.values,
    "Longitude": storm_2.lon.values,
    "Wind Speed (wmo)": storm_2.wmo_wind.values,
    "USA Status (wmo)": storm_2.usa_status.values
}

df_storm_1 = pd.DataFrame(storm_1_data)
df_storm_2 = pd.DataFrame(storm_2_data)

df_storm_1.head(50)

,Timestamp,Latitude,Longitude,Wind Speed (wmo),USA Status (wmo)
0,2005-06-08 18:00:00,16.900000,-84.000000,25.0,b'TD'
1,2005-06-08 21:00:00,17.120007,-83.942520,NaN,b'TD'
2,2005-06-09 00:00:00,17.400000,-83.900002,30.0,b'TD'
3,2005-06-09 03:00:00,17.777500,-83.885017,NaN,b'TD'
4,2005-06-09 06:00:00,18.200001,-83.900002,35.0,b'TS'
5,2005-06-09 09:00:00,18.607500,-83.942467,NaN,b'TS'
6,2005-06-09 12:00:00,19.000000,-84.000000,35.0,b'TS'
7,2005-06-09 15:00:00,19.357494,-84.050026,NaN,b'TS'
8,2005-06-09 18:00:00,19.700001,-84.099998,35.0,b'TS'
9,2005-06-09 21:00:00,20.042492,-84.142586,NaN,b'TS'


In [39]:
#Handling timestamps in NetCDF (native format of dataset) 
timestamps = storm_2.iso_time.values  # Adjust the variable name if different

# Convert byte strings to pandas datetime, handling missing values
timestamps = pd.to_datetime([t.decode("utf-8") if isinstance(t, bytes) else t for t in timestamps], errors="coerce")

# Drop NaN values
timestamps = timestamps.dropna()

# Extract month and day
years = timestamps.year
months = timestamps.month
days = timestamps.day

# Compute the average month and average day
avg_year = round(np.nanmean(years))
avg_month = round(np.nanmean(months))
avg_day = round(np.nanmean(days))

print(f"Average Storm Time: Year: {avg_year}, Month: {avg_month}, Day: {avg_day}")

anomaly_value = obs.loc[(obs["YR"] == avg_year) & (obs["MON"] == avg_month), "ANOM"]
#season_value = obs.loc[(obs["YR"] == avg_year) & (obs["MON"] == avg_month), "SEAS"]

if not anomaly_value.empty:
    print(f"during this season, the anomaly value was: {anomaly_value.values}")

Average Storm Time: Year: 2005, Month: 6, Day: 29
during this season, the anomaly value was: [0.11]


In [28]:
obs

,Unnamed: 0,YR,MON,TOTAL,ANOM,STATE
0,0,1950,1,24.72,-1.53,cold
1,1,1950,2,25.17,-1.34,cold
2,2,1950,3,25.75,-1.16,cold
3,3,1950,4,26.12,-1.18,cold
4,4,1950,5,26.32,-1.07,cold
...,...,...,...,...,...,...
895,895,2024,8,26.85,-0.11,neutral
896,896,2024,9,26.55,-0.21,neutral
897,897,2024,10,26.45,-0.26,neutral
898,898,2024,11,26.30,-0.37,neutral
